In [1]:
import torch
from transformers import pipeline, AutoTokenizer, set_seed
from transformers import GPT2Tokenizer, GPT2Model

/opt/conda/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# specifying device_map="auto" will be good enough as 🤗 Accelerate will attempt to fill all the space in your GPU(s), then loading them to the CPU, 
# and finally if there is not enough RAM it will be loaded to the disk

pipe = pipeline('text-generation', model='gpt2', device_map="auto")
output = pipe("This is a cool example!", do_sample=True, top_p=0.95, max_length=30, truncation=True, num_return_sequences=5)
print(output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [14]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
sequence = "hello world"
encoded_input = tokenizer(sequence, return_tensors='pt')
print(encoded_input)

model = GPT2Model.from_pretrained('gpt2')
output = model(**encoded_input)
# print(output)

{'input_ids': tensor([[31373,   995]]), 'attention_mask': tensor([[1, 1]])}


In [15]:
decoded_sequence = tokenizer.decode(encoded_input["input_ids"][0].tolist())
print(decoded_sequence)

hello world


# Trainer

In [6]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification
from datasets import load_dataset
import os

def tokenize_dataset(dataset):
    return tokenizer(dataset["text"])

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased")
os.environ["WANDB_API_KEY"] = "4560c294051a3d1f5a575d4d33347931c18dbfb5"

training_args = TrainingArguments(
    output_dir="training_output",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    report_to="wandb",  # enable logging to W&B
)

dataset = load_dataset("rotten_tomatoes") 
dataset = dataset.map(tokenize_dataset, batched=True) # use this map function
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# combine all 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)  # doctest: +SKIP


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 1066/1066 [00:00<00:00, 21286.93 examples/s]


In [5]:
# You can customize the training loop behavior by subclassing the methods inside Trainer. 
# This allows you to customize features such as the loss function, optimizer, and scheduler. 
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nexa4ai (nexaai). Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.450700
1000,0.393200
1500,0.267800
2000,0.282200


TrainOutput(global_step=2134, training_loss=0.3402043586595101, metrics={'train_runtime': 60.7028, 'train_samples_per_second': 281.041, 'train_steps_per_second': 35.155, 'total_flos': 195974132394480.0, 'train_loss': 0.3402043586595101, 'epoch': 2.0})

# Train in native PyTorch
https://huggingface.co/docs/transformers/training#train-with-pytorch-trainer

In [29]:
# dataLoader

from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# step 1 read in datasets
dataset = load_dataset("yelp_review_full")
dataset = DatasetDict({
    "train": dataset["train"].select(range(1000)),
    "test": dataset["test"].select(range(1000))
})

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
print(tokenized_datasets)

tokenized_datasets.set_format("torch") # Set the format of the dataset to return PyTorch tensors instead of lists

# Create a DataLoader for your training and test datasets so you can iterate over batches of data:
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8)
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=8)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 2964.30 examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})


In [32]:
# Optimizer and learning rate scheduler

from torch.optim import AdamW
from transformers import get_scheduler

num_epochs = 2
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [33]:
# training loop

from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

model.train()
for _ in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)


100%|██████████| 375/375 [01:06<00:00,  5.61it/s]

In [26]:
# evaluate

import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.151274,0.501000
2,No log,1.087506,0.536000


TrainOutput(global_step=250, training_loss=1.19839111328125, metrics={'train_runtime': 65.2303, 'train_samples_per_second': 30.661, 'train_steps_per_second': 3.833, 'total_flos': 526236284928000.0, 'train_loss': 1.19839111328125, 'epoch': 2.0})

# Fetch models and tokenizers to use offline

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig

tokenizer = AutoTokenizer.from_pretrained("./your/path/bigscience_t0")
model = AutoModel.from_pretrained("./your/path/bigscience_t0")

from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="bigscience/T0_3B", filename="config.json", cache_dir="./bigscience_t0")

config = AutoConfig.from_pretrained("./your/path/bigscience_t0/config.json")

# Pipeline

In [ ]:
from transformers import pipeline

transcriber = pipeline(task="automatic-speech-recognition")
transcriber("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")

# Load pretrained instances with an AutoClass
1. Text, use a Tokenizer to convert text into a sequence of tokens  
2. Speech and audio, use a Feature extractor to extract sequential features from audio waveforms and convert them into tensors.  
3. Image inputs use a ImageProcessor to convert images into tensors.  
4. Multimodal inputs, use a Processor to combine a tokenizer and a feature extractor or image processor.  

- Load a pretrained tokenizer.  
- Load a pretrained image processor  
- Load a pretrained feature extractor.  
- Load a pretrained processor.  
- Load a pretrained model.  
- Load a model as a backbone.  

In [6]:
# tokenizer
"""
input_ids are the indices corresponding to each token in the sentence.
attention_mask indicates whether a token should be attended to or not.
token_type_ids identifies which sequence a token belongs to when there is more than one sequence.
"""
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
sequence = "In a hole in the ground there lived a hobbit."
encoded_input = tokenizer(sequence)
print(encoded_input)

{'input_ids': [101, 1999, 1037, 4920, 1999, 1996, 2598, 2045, 2973, 1037, 7570, 10322, 4183, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [8]:
# the tokenizer added two special tokens - CLS and SEP (classifier and separator) - to the sentence. 
tokenizer.decode(encoded_input["input_ids"])

'[CLS] in a hole in the ground there lived a hobbit. [SEP]'

### padding token
Padding is a strategy for ensuring tensors are rectangular by adding a special padding token to shorter sentences.


In [9]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_input = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)
print(encoded_input)

{'input_ids': tensor([[  101,  2021,  2054,  2055,  2117,  6350,  1029,   102,     0,     0,
             0,     0,     0,     0],
        [  101,  2123,  1005,  1056,  2228,  2002,  4282,  2055,  2117,  6350,
          1010, 28315,  1012,   102],
        [  101,  2054,  2055,  5408, 14625,  1029,   102,     0,     0,     0,
             0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}


# Parameter-Efficient Fine Tuning (PEFT) 
Parameter-Efficient Fine Tuning (PEFT) methods freeze the pretrained model parameters during fine-tuning and add a small number of trainable parameters (the adapters) on top of it.  

In [35]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer
from peft import PeftConfig

model_id = "facebook/opt-350m"
adapter_model_id = "ybelkada/opt-350m-lora"
tokenizer = AutoTokenizer.from_pretrained(model_id)
text = "Hello"
inputs = tokenizer(text, return_tensors="pt")

model = AutoModelForCausalLM.from_pretrained(model_id)
peft_config = PeftConfig.from_pretrained(adapter_model_id)

# to initiate with random weights
peft_config.init_lora_weights = False

model.add_adapter(peft_config)
model.enable_adapters()

# Convert the output tensor to a list of integers
output_ids = output[0].tolist()  # Assuming you want the first generated sequence

# Decode the list of token IDs to a string
decoded_text = tokenizer.decode(output_ids, skip_special_tokens=True)
print(decoded_text)

tokenizer_config.json: 100%|██████████| 685/685 [00:00<00:00, 1.29MB/s]
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 11.1MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 91.5MB/s]
special_tokens_map.json: 100%|██████████| 441/441 [00:00<00:00, 710kB/s]
/opt/conda/envs/pytorch/lib/python3.10/site-packages/transformers/generation/utils.py:1183: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


# Generation with LLM best practices
https://huggingface.co/docs/transformers/llm_tutorial

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True
)
# Since LLMs are not trained to continue from pad tokens, your input needs to be left-padded.
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")
model_inputs = tokenizer(["A list of colors: red, blue"], return_tensors="pt").to("cuda")
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token  # Most LLMs don't have a pad token by default
model_inputs = tokenizer(
    ["A list of colors: red, blue", "Portugal is"], return_tensors="pt", padding=True
).to("cuda")
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)